In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

## get some data for test

In [ ]:
class getDataMetric(metrics.BaseMetric):
    """
    returns needed data 
    """
    
    def __init__(self, colname=['airmass', 'expMJD'], dataout=True, returnIdx=1, **kwargs):
        self.colname = colname
        self.returnIdx = returnIdx
        self.dataout = dataout
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
        
        
    def run(self, dataSlice, slicePoint=None):
        
        # return dataSlice directly
        if self.dataout:
            result = dataSlice
            return result
        else:
            result = np.mean(dataSlice[self.colname[int(self.returnIdx)]])
            return float(result)


In [ ]:
# run the metric

metric = getDataMetric(colname=['observationStartMJD', 'fiveSigmaDepth', 'filter'])

# ra,dec of ipix 
ra = np.array([255.9375])
dec = np.array([0.0])
slicer = slicers.UserPointsSlicer(ra, dec)

sqlconstraint = 'night<300 and proposalId=3'
metricSky = metricBundles.MetricBundle(metric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'metricSky':metricSky}, opsdb_baseline, outDir=outDir, resultsDb=resultsDb)
group.runAll()


## test the phase space metric 

In [ ]:
class phspaceMetric(metrics.BaseMetric):
    """
    returns needed data 
    """
    
    def __init__(self, colname=['airmass', 'expMJD'], dataout=True, returnIdx=1, **kwargs):
        self.colname = colname
        self.returnIdx = returnIdx
        self.dataout = dataout
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
        
        
    def run(self, dataSlice, slicePoint=None):
        
        # return dataSlice directly
        if self.dataout:
            result = dataSlice
            return result
        else:
            result = np.mean(dataSlice[self.colname[int(self.returnIdx)]])
            return float(result)
